# Symbiotic Relationships Simulation

In [ ]:
from mesa.discrete_space import CellAgent, FixedAgent
import math
import mesa
from mesa import Model
from mesa.datacollection import DataCollector
from mesa.discrete_space import OrthogonalMooreGrid, CellAgent, FixedAgent
from mesa.experimental.devs import ABMSimulator
import matplotlib.pyplot as plt

SEED = 42

class Animal(CellAgent):
    def __init__(self, model, initial_energy=50, p_reproduce=0.04, energy_from_food=4 , cell=None):
        super().__init__(model)
        self.cell = cell
        self.energy = initial_energy
        self.p_reproduce = p_reproduce
        self.energy_from_food = energy_from_food
        
    def feed(self):
        """Abstract method to be implemented by subclasses."""
        
    def step(self):
        self.energy -= 1
        if self.energy <= 0:
            self.remove()
            return

        self.move()
        self.feed()
        
        if self.random.random() < self.p_reproduce:
            self.reproduce()
    
    def move(self):
        self.cell=self.cell.neighborhood.select_random_cell()
        
    def reproduce(self):
        # """Abstract method to be implemented by subclasses."""
        self.energy /= 2
        self.__class__(
            model = self.model,
            initial_energy = self.energy,
            p_reproduce = self.p_reproduce,
            energy_from_food = self.energy_from_food,
            cell = self.cell,
        )


        
class Frog(Animal):
    def feed(self):
        """If possible, eat an ant at current location."""
        ant = [obj for obj in self.cell.agents if isinstance(obj, Ant)]
        if ant:  # If there are any ant present
            ant_to_eat = self.random.choice(ant)
            self.energy += self.energy_from_food
            ant_to_eat.remove()

    def move(self):
        """Move to a neighboring cell, preferably one with ant."""
        cells_with_ant = self.cell.neighborhood.select(
            lambda cell: any(isinstance(obj, Ant) for obj in cell.agents)
        )
        target_cells = (
            cells_with_ant if len(cells_with_ant) > 0 else self.cell.neighborhood
        )
        self.cell = target_cells.select_random_cell()
    
    # def reproduce(self):
        
    #     # self.energy /= 2
    #     self.__class__(
    #         model = self.model,
    #         initial_energy = self.energy,
    #         # self.p_reproduce,
    #         # self.energy_from_food,
    #         cell = self.cell,
    #     )
        

        
class Spider(Animal):
    def feed(self):
        """If possible, eat a frog at current location."""
        snake = [obj for obj in self.cell.agents if isinstance(obj, Snake)]
        if snake:  # If there are any frog present
            snake_to_eat = self.random.choice(snake)
            self.energy += self.energy_from_food
            snake_to_eat.remove()

    def move(self):
        """Move to a neighboring cell, preferably one with frog."""
        cells_with_snake = self.cell.neighborhood.select(
            lambda cell: any(isinstance(obj, Snake) for obj in cell.agents)
        )
        target_cells = (
            cells_with_snake if len(cells_with_snake) > 0 else self.cell.neighborhood
        )
        self.cell = target_cells.select_random_cell()

class Ant(Animal):
    pass    
    

class Snake(Animal):
    def feed(self):
        """If possible, eat a frog at current location."""
        frog = [obj for obj in self.cell.agents if isinstance(obj, Frog)]
        if frog:  # If there are any frog present
            frog_to_eat = self.random.choice(frog)
            self.energy += self.energy_from_food
            frog_to_eat.remove()

    def move(self):
        """Move to a neighboring cell, preferably one with frog."""
        cells_with_sheep = self.cell.neighborhood.select(
            lambda cell: any(isinstance(obj, Frog) for obj in cell.agents)
        )
        target_cells = (
            cells_with_sheep if len(cells_with_sheep) > 0 else self.cell.neighborhood
        )
        self.cell = target_cells.select_random_cell()



In [ ]:
class SymbioticRelationshipsModel(Model):
    def __init__(self,
                 width=32,
                 height=32,
                 initial_frogs=10,
                 initial_spiders=10,
                 initial_snakes=10,
                 initial_ants=10,
                 seed = None,
                 rng = None,
                 simulator: ABMSimulator = None):
        super().__init__(seed=seed, rng=rng)
        
        if simulator is None:
            simulator = ABMSimulator()

        self.simulator = simulator
        self.simulator.setup(self)
        
        self.height = height
        self.width = width
        
        # Create grid using experimental cell space
        self.grid = OrthogonalMooreGrid(
            [self.height, self.width],
            torus=False, # Decide upon this!?
            capacity=math.inf, #
            random=self.random,
        )
        
        # Set up data collection
        model_reporters = {
            "Spiders": lambda m: len(m.agents_by_type[Spider]),
            "Frogs": lambda m: len(m.agents_by_type[Frog]),
        }

        self.datacollector = DataCollector(model_reporters)
        
        
        Frog.create_agents(
            self,
            initial_frogs, # frog amount
            cell=self.random.choices(self.grid.all_cells.cells, k=initial_frogs),
        )

        Snake.create_agents(
            self,
            initial_snakes, # Snake amount
            cell=self.random.choices(self.grid.all_cells.cells, k=initial_snakes),
        )
        
        Spider.create_agents(
            self,
            initial_spiders, # Spider amount
            cell=self.random.choices(self.grid.all_cells.cells, k=initial_spiders),
        )

        Ant.create_agents(
            self,
            initial_ants, # Ant amount
            cell=self.random.choices(self.grid.all_cells.cells, k=initial_ants),
        )
        
        # Collect initial data
        self.running = True
        self.datacollector.collect(self)
        
    def step(self):
        """Execute one step of the model."""
        self.agents_by_type[Ant].shuffle_do("step")
        self.agents_by_type[Snake].shuffle_do("step")
        self.agents_by_type[Frog].shuffle_do("step")
        self.agents_by_type[Spider].shuffle_do("step")
        
        # Collect data
        self.datacollector.collect(self)
        
        # Spawn ants every 2 ticks
        if self.steps % 2 == 0:
            Ant.create_agents(
                self,
                1, # Ant amount
                cell=self.grid.select_random_empty_cell()) # Might spawn at unlucky place (against nest), will that be a problem?!?!
            
        



In [ ]:
# use Rectangle to draw background zones using patches
from matplotlib.patches import Rectangle
import solara

@solara.component
def CustomSpaceVisualization(model):
    """Custom space visualization with colored background zones"""
    # This is required to update the visualization when the model steps
    from mesa.visualization.components.matplotlib_components import update_counter
    update_counter.get()  # This triggers re-rendering on model updates
    # Setup figure
    fig = plt.Figure(figsize=(8, 8))    # Note: you must initialize a figure using this method instead of
    ax = fig.subplots()
    # set the width and height based on model
    width, height = model.width, model.height
    # Draw home area (light blue background)

    
    # # ax.add_patch(default_environment_patch)
    # Draw hardcoded nests TODO: fetch from model dynamically, this way we can code with it (agents have access to the zone info)
    nest1 = Rectangle((4, 4), 3, 3,
                         linewidth=0, facecolor='lightcoral', alpha=0.2)
    ax.add_patch(nest1)
    
    nest2 = Rectangle((26, 4), 3, 3,
                         linewidth=0, facecolor='lightcoral', alpha=0.2)
    ax.add_patch(nest2)
    
    nest3 = Rectangle((4, 26), 3, 3,
                         linewidth=0, facecolor='lightcoral', alpha=0.2)
    ax.add_patch(nest3)
    
    nest4 = Rectangle((26, 26), 3, 3,
                         linewidth=0, facecolor='lightcoral', alpha=0.2)
    ax.add_patch(nest4)
    
    # Draw our agents
    for frog in model.agents_by_type[Frog]:
        x, y = frog.cell.coordinate
        
        color = "tab:green"
        marker = "o"
        size = 100
        #     # Use matplotlib's OOP API instead of plt.scatter for thread safety
        ax.scatter(x + 0.5, y + 0.5, c=color, s=size, marker=marker, zorder=10, alpha=0.8)
        
    for spider in model.agents_by_type[Spider]:
        x, y = spider.cell.coordinate
        
        color = "tab:brown"
        marker = "X"
        size = 100
        #     # Use matplotlib's OOP API instead of plt.scatter for thread safety
        ax.scatter(x + 0.5, y + 0.5, c=color, marker=marker, s=size, zorder=10, alpha=0.8)
        
    for ant in model.agents_by_type[Ant]:
        x, y = ant.cell.coordinate
        
        color = "tab:red"
        marker = "d"
        size = 100
        #     # Use matplotlib's OOP API instead of plt.scatter for thread safety
        ax.scatter(x + 0.5, y + 0.5, c=color, marker=marker, s=size, zorder=10, alpha=0.8)
        
    for snake in model.agents_by_type[Snake]:
        x, y = snake.cell.coordinate
        
        color = "tab:orange"
        marker = "s"
        size = 100

            
    #     # Use matplotlib's OOP API instead of plt.scatter for thread safety
        ax.scatter(x + 0.5, y + 0.5, c=color, marker=marker, s=size, zorder=10, alpha=0.8)
    
    # Set up the plot
    ax.set_xlim(0, width)
    ax.set_ylim(0, height)
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.3)
    ax.set_xlabel('X coordinate')
    ax.set_ylabel('Y coordinate')
    
    # This is required to render the visualization
    solara.FigureMatplotlib(fig)

In [ ]:
from mesa.visualization import (
    CommandConsole,
    Slider,
    SolaraViz,
    make_plot_component,
    make_space_component,
)

# This function will be called for every agent present in the model and defines a visualization (color, marker...)
def agent_portrayal(agent):
    if agent is None:
        return

    portrayal = {
        # "size": 25,
    }


    # Filter the agent type, then describe the agent visuals
    if isinstance(agent, Frog):
        portrayal["color"] = "tab:green"
        portrayal["marker"] = "o"
        portrayal["zorder"] = 2

    elif isinstance(agent, Spider):
        portrayal["color"] = "tab:brown"
        portrayal["marker"] = "X"
        portrayal["zorder"] = 2

    elif isinstance(agent, Ant):
        portrayal["color"] = "tab:red"
        portrayal["marker"] = "d"
        portrayal["zorder"] = 2

    elif isinstance(agent, Snake):
        portrayal["color"] = "tab:orange"
        portrayal["marker"] = "s"
        portrayal["zorder"] = 2
    

    return portrayal


model_params = {
    "seed": {
        "type": "InputText",
        "value": SEED,
        "label": "Random Seed",
    }
}

def post_process_space(ax):
    ax.set_aspect("equal")
    ax.set_xticks([])
    ax.set_yticks([])

# The main grid with all the agents
space_component = make_space_component(
    agent_portrayal, draw_grid=False, post_process=post_process_space
)

def post_process_lines(ax):
    ax.legend(loc="center left", bbox_to_anchor=(1, 0.9))

simulator = ABMSimulator() 
model = SymbioticRelationshipsModel(seed=SEED, simulator=simulator)

lineplot_component = make_plot_component(
    {"Frogs": "tab:green", "Spiders": "tab:brown"},
    post_process=post_process_lines,
)

page = SolaraViz(
    model,
    # components=[space_component, lineplot_component, CommandConsole],
    components=[space_component, CustomSpaceVisualization, lineplot_component, CommandConsole],
    model_params=model_params,
    name="Symbiotic Relationships",
    simulator=simulator,
)

page  # noqa